# Overview

This noteboook will sniff your phone for a designated text message. The moment that your mother texts you with the keywork "GameStop", this script will log into your Robinhood account and sell all of your assets.

Dependencies
- Must be run on a MacOS device to sniff through iMessage

In [1]:
pip install robin_stocks

   ---------------------------------------- 0.0/133.3 kB ? eta -:--:--
   ----------- --------------------------- 41.0/133.3 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 133.3/133.3 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Global Variables

In [ ]:
target_sender = 'Mom'
keywords = ['GME','GameStop','AMC']

# Robinhood Code

In [3]:
import robin_stocks.robinhood as r

def sell_all_stocks():
    # Get all owned stocks
    portfolio = r.build_holdings()

    for symbol, data in portfolio.items():
        quantity = data['quantity']
        if float(quantity) > 0:
            # Place a market sell order for each stock
            r.orders.order_sell_market(symbol, quantity)

def sell_all_options():
    options = r.options.get_open_option_positions()

    for option in options:
        option_id = option['id']
        quantity = option['quantity']
        if float(quantity) > 0:
            r.options.order_sell_to_close(option_id, quantity)

def sell_all_crypto():
    cryptos = r.crypto.get_crypto_positions()

    for crypto in cryptos:
        symbol = crypto['currency']['code']
        quantity = crypto['quantity']
        if float(quantity) > 0:
            r.orders.order_sell_crypto_by_quantity(symbol, quantity)


# Usage

# Log in to Robinhood
def sell_everything():
    username = 'your_username'
    password = 'your_password'
    r.login(username, password)
    
    sell_all_stocks()
    sell_all_options()
    sell_all_crypto()
    
    r.logout()


Exception: Unable to log in with provided credentials.

# Sniff through iMessage

In [ ]:
import subprocess
import time

# Function to execute AppleScript and get the latest message
def get_latest_imessage():
    script = '''
    tell application "Messages"
        set latestMessage to a reference to the first item of (get messages of text chat 1)
        set messageText to latestMessage's text
        set senderName to latestMessage's sender's name
        return senderName & ": " & messageText
    end tell
    '''
    try:
        result = subprocess.run(['osascript', '-e', script], capture_output=True, text=True)
        return result.stdout.strip()
    except Exception as e:
        print(f"Error: {e}")
        return None

# Previous message to compare with new messages
previous_message = ""

# Check for new messages every second
while True:
    latest_message = get_latest_imessage()
    if latest_message and latest_message != previous_message:
        sender, message = latest_message.split(': ', 1)
        
        print(f"New message received from {sender}: {message}")
        previous_message = latest_message
        print(f"Sender stored as string: {sender}")

        if sender == target_sender and keywords in previous_message():
            sell_everything()
    time.sleep(1)
